# Проект для «Викишоп»

<a id=0></a>
[Содержание](#0)

[1. Описание проекта](#1)

[2. Анализ данных](#2)

[3. Подготовка данных](#3)

[4. Обучение моделей](#4)

[5. Тестирование модели](#5)

[6. Общий вывод](#6)

<a id=1></a>
## 1. Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель проекта:**

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Данные:**

Данные находятся в файле:

`/datasets/toxic_comments.csv`

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

**План работы:**

1. Загрузить данные.

2. Проанализировать данные, заполнить пропущенные значения.

3. Подготовить выборки для обучения моделей.

4. Обучить разные модели с различными гиперпараметрами.

5. Опираясь на критерии заказчика, выбрать лучшую модель, проверить её качество на тестовой выборке.

6. Сделать вывод.

Для выполнения проекта можно использовать модель *BERT*.

<a id=2></a>
[Содержание](#0)
## 2. Анализ данных

In [1]:
import pandas as pd
import numpy as np
import datetime
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

RND_STATE = 20032023

ModuleNotFoundError: No module named 'sklearn'

Загрузим список стоп-слов:

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Загрузим разметчик частей речи:

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Загрузим данные из файла в датафрейм:

In [4]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col = [0])
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col = [0])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков данных нет.

Посмотрим соотношение 0 и 1 в целевом признаке `toxic`:

In [5]:
data.toxic.value_counts() / data.shape[0] * 100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

Сохраним соотношение классов в переменную для дальнейшего использования, при обучении моделей:

In [6]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

8.841344371679229

Напишем функцию предобработки текста, которая выполняет следующие действия:
- Оставляет только латиницу.
- Переводит в нижний регистр.
- Удаляет слова из stop_words.

In [7]:
def clear_text(text):
    stop_words = set(nltk_stopwords.words('english'))
    text = text.lower()
    word_list = re.sub(r'[^a-z ]', ' ', text).split()
    word_not_stop_list = [w for w in word_list if not w in stop_words]
    return ' '.join(word_not_stop_list)

In [8]:
data['text_clear'] = data['text'].progress_apply(clear_text)
display(data)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 159292/159292 [00:10<00:00, 15870.73it/s]


,text,toxic,text_clear
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww matches background colour seemingly stuck ...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,second time asking view completely contradicts...
159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed horrible thing put talk page
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres actual article prostitution...
159449,And it looks like it was actually you who put ...,0,looks like actually put speedy first version d...


Подготовим два набора данных:
- Без учёта части речи.
- С учётом части речи (POS-теги).

Для лематизации строк спользуем WordNet Lemmatizer:

In [9]:
WNL = WordNetLemmatizer()

Лемматизируем строку с WordNetLemmatizer и сохраним в столбец `text_wnl`:

In [10]:
def lemmatize_text(text):

    word_list = text.split()
    lemmatized_text = ' '.join([WNL.lemmatize(w) for w in word_list])

    return lemmatized_text

In [11]:
start_time = datetime.datetime.now()
data['text_wnl'] = data['text_clear'].progress_apply(lemmatize_text)
print('Время обработки:',(datetime.datetime.now() - start_time).seconds, 'с')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 159292/159292 [00:10<00:00, 15131.40it/s]

Время обработки: 10 с


Лемматизируем строку с WordNetLemmatizer с учетом nltk.pos_tag и сохраним в столбец `text_wnl_postag`:

In [12]:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
def lemmatize_postag_text(text):

    word_list = text.split()
    lemmatized_text = ' '.join([WNL.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    
    return lemmatized_text

In [14]:
start_time = datetime.datetime.now()
data['text_wnl_postag'] = data['text_clear'].progress_apply(lemmatize_postag_text)
print('Время обработки:',(datetime.datetime.now() - start_time).seconds, 'с')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 159292/159292 [05:32<00:00, 479.66it/s]

Время обработки: 332 с


In [15]:
display(data)

,text,toxic,text_clear,text_wnl,text_wnl_postag
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edits made username hardcore metal...,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww matches background colour seemingly stuck ...,aww match background colour seemingly stuck th...,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...,hey man really trying edit war guy constantly ...,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...,make real suggestion improvement wondered sect...,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page,sir hero chance remember page,sir hero chance remember page
...,...,...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,second time asking view completely contradicts...,second time asking view completely contradicts...,second time ask view completely contradicts co...
159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed horrible thing put talk page,ashamed horrible thing put talk page,ashamed horrible thing put talk page
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres actual article prostitution...,spitzer umm there actual article prostitution ...,spitzer umm there actual article prostitution ...
159449,And it looks like it was actually you who put ...,0,looks like actually put speedy first version d...,look like actually put speedy first version de...,look like actually put speedy first version de...


**Вывод:**

В этом разделе были выполнены следующие задачи:

- Комментарии очищены: буквы переведены в нижний регистр, оставлена только латиница, удалены стоп-слова.
- Комментарии лемматизированы без учёта части речи и с учётом части речи (POS-тегов).

<a id=3></a>
[Содержание](#0)
## 3. Подготовка данных

Подготовим наборы данных для обучения (без учёта части речи и с учётом части речи) и целевой признак:

In [16]:
corpus = data['text_wnl']
corpus_postag = data['text_wnl_postag']
y = data['toxic']

In [17]:
x_train, x_test, y_train, y_test = train_test_split(corpus, y,
     test_size = 0.2, stratify = y, shuffle = True, random_state = RND_STATE)

x_tag_train, x_tag_test, y_tag_train, y_tag_test = train_test_split(corpus_postag, y,
     test_size = 0.2, stratify = y, shuffle = True, random_state = RND_STATE)

print('Размер тренерующей выборки:', x_train.shape)
print('Размер тестовой выборки:', x_test.shape)
print('Размер тренерующей выборки postag:', x_tag_train.shape)
print('Размер тестовой выборки postag:', x_tag_test.shape)

Размер тренерующей выборки: (127433,)
Размер тестовой выборки: (31859,)
Размер тренерующей выборки postag: (127433,)
Размер тестовой выборки postag: (31859,)


In [18]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(x_train)
tf_idf_test = count_tf_idf.transform(x_test)

count_tag_tf_idf = TfidfVectorizer()
tf_idf_tag_train = count_tag_tf_idf.fit_transform(x_tag_train)
tf_idf_tag_test = count_tag_tf_idf.transform(x_tag_test)

print('Размер тренерующей выборки tf_idf:', tf_idf_train.shape)
print('Размер тестовой выборки tf_idf:', tf_idf_test.shape)
print('Размер тренерующей выборки tf_idf postag:', tf_idf_tag_train.shape)
print('Размер тестовой выборки tf_idf postag:', tf_idf_tag_test.shape)

Размер тренерующей выборки tf_idf: (127433, 138968)
Размер тестовой выборки tf_idf: (31859, 138968)
Размер тренерующей выборки tf_idf postag: (127433, 132779)
Размер тестовой выборки tf_idf postag: (31859, 132779)


<a id=4></a>
[Содержание](#0)
## 4. Обучение моделей

Обучим следующие модели:

- LogisticRegression
- DecisionTreeClassifier
- LightGBM
- SGDClassifier
- CatBoostClassifier

Сохраним результаты в таблицу scores_data.

Зададим общие параметры для кроссвалидации.

Напишем функцию для подбора гиперпараметров для моделей с разными наборами данных (`text_wnl` или `text_wnl_postag`):

In [19]:
scores_data = []

In [20]:
kfold = KFold(n_splits = 8, random_state = RND_STATE, shuffle = True)

In [21]:
def fit_model(estimator, param_grid, param):
   
    model = GridSearchCV(
                         estimator = estimator,
                         param_grid = param_grid,
                         n_jobs = 4,
                         cv = kfold,
                         scoring = 'f1',
                        )

    if param == 'text_wnl':
        x = tf_idf_train
    else:
        x = tf_idf_tag_train
        
    model.fit(x, y_train)
    
    res = pd.DataFrame(model.cv_results_).iloc[model.best_index_]
    f1 = round(model.cv_results_['mean_test_score'][model.best_index_], 4)
    fit_time = round(res['mean_fit_time'], 3)
    score_time = round(res['mean_score_time'], 3)
    
#    print("Лучший набор параметров:", model.best_params_)
    print('Время обучения модели:', fit_time, 's')
    print('Время предсказания модели:', score_time, 's')
    print('Значение f1:', f1)

    scores_data.append([estimator, f1, fit_time, score_time, param, model.best_params_])
    
    return model.best_estimator_, model.best_score_

**4.1. LogisticRegression**

In [23]:
param_grid = {'C' : [0.01, 0.1, 1, 5, 10],
              'solver' : ['lbfgs', 'liblinear'],
              'max_iter' : [1000],
              'class_weight' : [{0 : 1, 1 : class_ratio}],
}

fit_model(LogisticRegression(), param_grid, 'text_wnl')

Время обучения модели: 4.644 s
Время предсказания модели: 0.004 s
Значение f1: 0.7688


(LogisticRegression(C=5, class_weight={0: 1, 1: 8.841344371679229},
                    max_iter=1000),
 0.7687578946125302)

In [24]:
param_grid = {'C' : [0.01, 0.1, 1, 5, 10],
              'solver' : ['lbfgs', 'liblinear'],
              'max_iter' : [1000],
              'class_weight' : [{0 : 1, 1 : class_ratio}],
}

fit_model(LogisticRegression(), param_grid, 'text_wnl_postag')

Время обучения модели: 1.206 s
Время предсказания модели: 0.004 s
Значение f1: 0.7651


(LogisticRegression(C=5, class_weight={0: 1, 1: 8.841344371679229},
                    max_iter=1000, solver='liblinear'),
 0.76506439603784)

**4.2. DecisionTreeClassifier**

In [25]:
param_grid = {
    'max_depth': range(5, 33, 2),
    'min_samples_leaf': [ 1, 2, 3],
    'class_weight' : [{0 : 1, 1 : class_ratio}],
}

fit_model(DecisionTreeClassifier(), param_grid, 'text_wnl')

Время обучения модели: 23.329 s
Время предсказания модели: 0.016 s
Значение f1: 0.6284


(DecisionTreeClassifier(class_weight={0: 1, 1: 8.841344371679229}, max_depth=27,
                        min_samples_leaf=2),
 0.6284314300513127)

In [26]:
param_grid = {
            'max_depth': range(5, 33, 2),
            'min_samples_leaf': [ 1, 2, 3],
            'class_weight' : [{0 : 1, 1 : class_ratio}],
            }

fit_model(DecisionTreeClassifier(), param_grid, 'text_wnl_postag')

Время обучения модели: 23.717 s
Время предсказания модели: 0.016 s
Значение f1: 0.65


(DecisionTreeClassifier(class_weight={0: 1, 1: 8.841344371679229}, max_depth=31,
                        min_samples_leaf=3),
 0.6500343295135234)

**4.3. LGBMClassifier**

In [27]:
param_grid = {
            'n_estimators': range(100, 351, 50),
            'learning_rate': [0.1, 0.3, 0.5],
            'max_depth': [5, 7, 9, 11, 13],
            'objective' : ['binary'],
            'class_weight' : [{0 : 1, 1 : class_ratio}],
            }

fit_model(LGBMClassifier(), param_grid, 'text_wnl')

Время обучения модели: 16.002 s
Время предсказания модели: 0.54 s
Значение f1: 0.7719


(LGBMClassifier(class_weight={0: 1, 1: 8.841344371679229}, learning_rate=0.3,
                max_depth=9, n_estimators=350, objective='binary'),
 0.771920708677636)

In [28]:
param_grid = {
            'n_estimators': range(100, 351, 50),
            'learning_rate': [0.1, 0.3, 0.5],
            'max_depth': [5, 7, 9, 11, 13],
            'objective' : ['binary'],
            'class_weight' : [{0 : 1, 1 : class_ratio}],
            }

fit_model(LGBMClassifier(), param_grid, 'text_wnl_postag')

Время обучения модели: 16.037 s
Время предсказания модели: 0.645 s
Значение f1: 0.7733


(LGBMClassifier(class_weight={0: 1, 1: 8.841344371679229}, learning_rate=0.3,
                max_depth=11, n_estimators=350, objective='binary'),
 0.7732601562305949)

**4.4. SGDClassifier**

In [29]:
param_grid = {
            'class_weight' : [{0 : 1, 1 : class_ratio}],
            'eta0' : [0.0, 0.1, 0.2],
            'learning_rate' : ['adaptive'],
            'loss': ['hinge', 'log_loss', 'modified_huber'],
            }

fit_model(SGDClassifier(), param_grid, 'text_wnl')

Время обучения модели: 1.193 s
Время предсказания модели: 0.003 s
Значение f1: 0.7612


(SGDClassifier(class_weight={0: 1, 1: 8.841344371679229}, eta0=0.1,
               learning_rate='adaptive', loss='modified_huber'),
 0.7611550471698018)

In [30]:
param_grid = {
            'class_weight' : [{0 : 1, 1 : class_ratio}],
            'eta0' : [0.0, 0.1, 0.2],
            'learning_rate' : ['adaptive'],
            'loss': ['hinge', 'log_loss', 'modified_huber'],
            }

fit_model(SGDClassifier(), param_grid, 'text_wnl_postag')

Время обучения модели: 1.083 s
Время предсказания модели: 0.004 s
Значение f1: 0.7589


(SGDClassifier(class_weight={0: 1, 1: 8.841344371679229}, eta0=0.2,
               learning_rate='adaptive', loss='modified_huber'),
 0.7588814359619608)

**4.5. CatBoostClassifier**

In [31]:
param_grid = {
            'depth': [5, 7, 9],
            'iterations': [200],
            'learning_rate': [0.1],
            }

fit_model(CatBoostClassifier(), param_grid, 'text_wnl')

0:	learn: 0.5942070	total: 803ms	remaining: 2m 39s
1:	learn: 0.5174332	total: 1.38s	remaining: 2m 16s
2:	learn: 0.4573324	total: 1.75s	remaining: 1m 54s
3:	learn: 0.4111333	total: 2.29s	remaining: 1m 52s
4:	learn: 0.3741757	total: 2.75s	remaining: 1m 47s
5:	learn: 0.3451169	total: 3.21s	remaining: 1m 43s
6:	learn: 0.3232627	total: 3.67s	remaining: 1m 41s
7:	learn: 0.3061020	total: 4.17s	remaining: 1m 40s
8:	learn: 0.2915861	total: 4.64s	remaining: 1m 38s
9:	learn: 0.2802528	total: 5.1s	remaining: 1m 36s
10:	learn: 0.2711173	total: 5.56s	remaining: 1m 35s
11:	learn: 0.2633234	total: 6.02s	remaining: 1m 34s
12:	learn: 0.2559116	total: 6.5s	remaining: 1m 33s
13:	learn: 0.2506412	total: 6.99s	remaining: 1m 32s
14:	learn: 0.2457685	total: 7.47s	remaining: 1m 32s
15:	learn: 0.2420022	total: 7.97s	remaining: 1m 31s
16:	learn: 0.2387977	total: 8.41s	remaining: 1m 30s
17:	learn: 0.2358039	total: 8.91s	remaining: 1m 30s
18:	learn: 0.2329300	total: 9.38s	remaining: 1m 29s
19:	learn: 0.2299583	tot

0:	learn: 0.5953497	total: 1.18s	remaining: 3m 54s
1:	learn: 0.5173883	total: 1.71s	remaining: 2m 49s
2:	learn: 0.4580651	total: 2.27s	remaining: 2m 28s
3:	learn: 0.4117895	total: 2.74s	remaining: 2m 14s
4:	learn: 0.3751885	total: 3.24s	remaining: 2m 6s
5:	learn: 0.3469690	total: 3.71s	remaining: 2m
6:	learn: 0.3249134	total: 4.16s	remaining: 1m 54s
7:	learn: 0.3069472	total: 4.63s	remaining: 1m 51s
8:	learn: 0.2923889	total: 5.09s	remaining: 1m 48s
9:	learn: 0.2803190	total: 5.57s	remaining: 1m 45s
10:	learn: 0.2709971	total: 6.02s	remaining: 1m 43s
11:	learn: 0.2632383	total: 6.5s	remaining: 1m 41s
12:	learn: 0.2568864	total: 6.99s	remaining: 1m 40s
13:	learn: 0.2504735	total: 7.47s	remaining: 1m 39s
14:	learn: 0.2461052	total: 7.96s	remaining: 1m 38s
15:	learn: 0.2420497	total: 8.4s	remaining: 1m 36s
16:	learn: 0.2379090	total: 8.93s	remaining: 1m 36s
17:	learn: 0.2351514	total: 9.39s	remaining: 1m 34s
18:	learn: 0.2324805	total: 9.84s	remaining: 1m 33s
19:	learn: 0.2300412	total: 1

0:	learn: 0.5948002	total: 483ms	remaining: 1m 36s
1:	learn: 0.5152359	total: 1.34s	remaining: 2m 12s
2:	learn: 0.4536128	total: 1.87s	remaining: 2m 3s
3:	learn: 0.4077218	total: 2.27s	remaining: 1m 51s
4:	learn: 0.3713700	total: 2.71s	remaining: 1m 45s
5:	learn: 0.3424208	total: 3.2s	remaining: 1m 43s
6:	learn: 0.3202121	total: 3.68s	remaining: 1m 41s
7:	learn: 0.3032546	total: 4.13s	remaining: 1m 39s
8:	learn: 0.2892068	total: 4.6s	remaining: 1m 37s
9:	learn: 0.2781611	total: 5.08s	remaining: 1m 36s
10:	learn: 0.2682788	total: 5.55s	remaining: 1m 35s
11:	learn: 0.2608633	total: 6.05s	remaining: 1m 34s
12:	learn: 0.2547395	total: 6.51s	remaining: 1m 33s
13:	learn: 0.2497297	total: 7.01s	remaining: 1m 33s
14:	learn: 0.2451560	total: 7.47s	remaining: 1m 32s
15:	learn: 0.2410618	total: 7.92s	remaining: 1m 31s
16:	learn: 0.2377074	total: 8.39s	remaining: 1m 30s
17:	learn: 0.2341368	total: 8.92s	remaining: 1m 30s
18:	learn: 0.2311215	total: 9.34s	remaining: 1m 28s
19:	learn: 0.2289740	tota

0:	learn: 0.5936343	total: 934ms	remaining: 3m 5s
1:	learn: 0.5144911	total: 1.46s	remaining: 2m 24s
2:	learn: 0.4540155	total: 1.93s	remaining: 2m 6s
3:	learn: 0.4086207	total: 2.43s	remaining: 1m 58s
4:	learn: 0.3722546	total: 2.87s	remaining: 1m 52s
5:	learn: 0.3430305	total: 3.31s	remaining: 1m 46s
6:	learn: 0.3208132	total: 3.81s	remaining: 1m 45s
7:	learn: 0.3042735	total: 4.27s	remaining: 1m 42s
8:	learn: 0.2901662	total: 4.74s	remaining: 1m 40s
9:	learn: 0.2788025	total: 5.18s	remaining: 1m 38s
10:	learn: 0.2695444	total: 5.67s	remaining: 1m 37s
11:	learn: 0.2621856	total: 6.13s	remaining: 1m 36s
12:	learn: 0.2544594	total: 6.62s	remaining: 1m 35s
13:	learn: 0.2495615	total: 7.08s	remaining: 1m 34s
14:	learn: 0.2450825	total: 7.55s	remaining: 1m 33s
15:	learn: 0.2411625	total: 7.91s	remaining: 1m 30s
16:	learn: 0.2379322	total: 8.47s	remaining: 1m 31s
17:	learn: 0.2350189	total: 8.87s	remaining: 1m 29s
18:	learn: 0.2323740	total: 9.33s	remaining: 1m 28s
19:	learn: 0.2296321	tot

0:	learn: 0.5969921	total: 814ms	remaining: 2m 41s
1:	learn: 0.5189616	total: 1.4s	remaining: 2m 18s
2:	learn: 0.4565339	total: 1.97s	remaining: 2m 9s
3:	learn: 0.4096035	total: 2.41s	remaining: 1m 58s
4:	learn: 0.3738266	total: 2.84s	remaining: 1m 50s
5:	learn: 0.3446820	total: 3.28s	remaining: 1m 46s
6:	learn: 0.3218712	total: 3.75s	remaining: 1m 43s
7:	learn: 0.3045587	total: 4.25s	remaining: 1m 42s
8:	learn: 0.2906054	total: 4.62s	remaining: 1m 38s
9:	learn: 0.2790970	total: 5.1s	remaining: 1m 36s
10:	learn: 0.2700328	total: 5.61s	remaining: 1m 36s
11:	learn: 0.2625549	total: 6.02s	remaining: 1m 34s
12:	learn: 0.2564450	total: 6.42s	remaining: 1m 32s
13:	learn: 0.2507205	total: 6.9s	remaining: 1m 31s
14:	learn: 0.2448651	total: 7.4s	remaining: 1m 31s
15:	learn: 0.2410782	total: 7.88s	remaining: 1m 30s
16:	learn: 0.2376517	total: 8.36s	remaining: 1m 29s
17:	learn: 0.2348602	total: 8.8s	remaining: 1m 28s
18:	learn: 0.2322802	total: 9.25s	remaining: 1m 28s
19:	learn: 0.2297162	total: 

...

23:	learn: 0.2032917	total: 6m 43s	remaining: 49m 20s
24:	learn: 0.2017448	total: 6m 56s	remaining: 48m 33s
25:	learn: 0.2001045	total: 7m 7s	remaining: 47m 43s
26:	learn: 0.1982570	total: 7m 23s	remaining: 47m 21s
27:	learn: 0.1969197	total: 7m 37s	remaining: 46m 53s
28:	learn: 0.1957738	total: 7m 52s	remaining: 46m 23s
29:	learn: 0.1946446	total: 8m 6s	remaining: 45m 56s
30:	learn: 0.1935726	total: 8m 21s	remaining: 45m 31s
31:	learn: 0.1922930	total: 8m 35s	remaining: 45m 5s
32:	learn: 0.1910837	total: 8m 49s	remaining: 44m 41s
33:	learn: 0.1894155	total: 9m 4s	remaining: 44m 18s
34:	learn: 0.1883963	total: 9m 18s	remaining: 43m 54s
35:	learn: 0.1874903	total: 9m 33s	remaining: 43m 30s
36:	learn: 0.1866465	total: 9m 47s	remaining: 43m 7s
37:	learn: 0.1857739	total: 10m 1s	remaining: 42m 45s
38:	learn: 0.1849368	total: 10m 16s	remaining: 42m 23s
39:	learn: 0.1841591	total: 10m 30s	remaining: 42m 1s
40:	learn: 0.1834525	total: 10m 43s	remaining: 41m 37s
41:	learn: 0.1827376	total: 10m

0:	learn: 0.5926577	total: 2.38s	remaining: 7m 54s
1:	learn: 0.5088710	total: 5.35s	remaining: 8m 49s
2:	learn: 0.4461228	total: 8.17s	remaining: 8m 56s
3:	learn: 0.3973766	total: 11s	remaining: 8m 58s
4:	learn: 0.3595540	total: 13.8s	remaining: 8m 59s
5:	learn: 0.3291277	total: 16.8s	remaining: 9m 2s
6:	learn: 0.3063411	total: 19.6s	remaining: 9m
7:	learn: 0.2885652	total: 22.5s	remaining: 8m 59s
8:	learn: 0.2737305	total: 25.4s	remaining: 8m 58s
9:	learn: 0.2621487	total: 28.3s	remaining: 8m 57s
10:	learn: 0.2524583	total: 31.2s	remaining: 8m 56s
11:	learn: 0.2438351	total: 33.9s	remaining: 8m 51s
12:	learn: 0.2376138	total: 36.8s	remaining: 8m 48s
13:	learn: 0.2321901	total: 39.6s	remaining: 8m 45s
14:	learn: 0.2277277	total: 42.5s	remaining: 8m 43s
15:	learn: 0.2237050	total: 45.3s	remaining: 8m 40s
16:	learn: 0.2201950	total: 48.2s	remaining: 8m 38s
17:	learn: 0.2169290	total: 51s	remaining: 8m 35s
18:	learn: 0.2136799	total: 54s	remaining: 8m 34s
19:	learn: 0.2107681	total: 56.8s

57:	learn: 0.1717748	total: 2m 45s	remaining: 6m 44s
58:	learn: 0.1711194	total: 2m 47s	remaining: 6m 40s
59:	learn: 0.1706733	total: 2m 50s	remaining: 6m 37s
60:	learn: 0.1702232	total: 2m 53s	remaining: 6m 34s
61:	learn: 0.1693917	total: 2m 55s	remaining: 6m 31s
62:	learn: 0.1687347	total: 2m 58s	remaining: 6m 28s
63:	learn: 0.1682380	total: 3m 1s	remaining: 6m 25s
64:	learn: 0.1677974	total: 3m 4s	remaining: 6m 22s
65:	learn: 0.1671822	total: 3m 6s	remaining: 6m 19s
66:	learn: 0.1666568	total: 3m 9s	remaining: 6m 16s
67:	learn: 0.1662421	total: 3m 12s	remaining: 6m 13s
68:	learn: 0.1657782	total: 3m 15s	remaining: 6m 10s
69:	learn: 0.1654005	total: 3m 17s	remaining: 6m 7s
70:	learn: 0.1649952	total: 3m 20s	remaining: 6m 3s
71:	learn: 0.1646097	total: 3m 22s	remaining: 6m
72:	learn: 0.1641406	total: 3m 25s	remaining: 5m 57s
73:	learn: 0.1637414	total: 3m 28s	remaining: 5m 54s
74:	learn: 0.1633142	total: 3m 30s	remaining: 5m 51s
75:	learn: 0.1629544	total: 3m 33s	remaining: 5m 48s
76:

103:	learn: 0.1532438	total: 4m 49s	remaining: 4m 27s
104:	learn: 0.1529635	total: 4m 52s	remaining: 4m 24s
105:	learn: 0.1526909	total: 4m 54s	remaining: 4m 21s
106:	learn: 0.1523264	total: 4m 57s	remaining: 4m 18s
107:	learn: 0.1519967	total: 4m 59s	remaining: 4m 15s
108:	learn: 0.1517235	total: 5m 2s	remaining: 4m 12s
109:	learn: 0.1514480	total: 5m 4s	remaining: 4m 9s
110:	learn: 0.1510797	total: 5m 7s	remaining: 4m 6s
111:	learn: 0.1507492	total: 5m 10s	remaining: 4m 3s
112:	learn: 0.1504841	total: 5m 12s	remaining: 4m
113:	learn: 0.1502782	total: 5m 15s	remaining: 3m 57s
114:	learn: 0.1499415	total: 5m 17s	remaining: 3m 55s
115:	learn: 0.1496518	total: 5m 20s	remaining: 3m 52s
116:	learn: 0.1494057	total: 5m 23s	remaining: 3m 49s
117:	learn: 0.1491221	total: 5m 25s	remaining: 3m 46s
118:	learn: 0.1489189	total: 5m 28s	remaining: 3m 43s
119:	learn: 0.1485380	total: 5m 30s	remaining: 3m 40s
120:	learn: 0.1483168	total: 5m 33s	remaining: 3m 37s
121:	learn: 0.1480896	total: 5m 36s	re

(<catboost.core.CatBoostClassifier at 0x2a852c0d0>, 0.7190066441650901)

In [32]:
param_grid = {
            'depth': [5, 7, 9],
            'iterations': [200],
            'learning_rate': [0.1],
            }

fit_model(CatBoostClassifier(), param_grid, 'text_wnl_postag') 

0:	learn: 0.5954005	total: 539ms	remaining: 1m 47s
1:	learn: 0.5151625	total: 1.04s	remaining: 1m 43s
2:	learn: 0.4544633	total: 1.49s	remaining: 1m 37s
3:	learn: 0.4061278	total: 1.96s	remaining: 1m 36s
4:	learn: 0.3687414	total: 2.4s	remaining: 1m 33s
5:	learn: 0.3397231	total: 2.82s	remaining: 1m 31s
6:	learn: 0.3168645	total: 3.25s	remaining: 1m 29s
7:	learn: 0.3003003	total: 3.66s	remaining: 1m 27s
8:	learn: 0.2859006	total: 4.1s	remaining: 1m 27s
9:	learn: 0.2744557	total: 4.57s	remaining: 1m 26s
10:	learn: 0.2638661	total: 5.02s	remaining: 1m 26s
11:	learn: 0.2564628	total: 5.47s	remaining: 1m 25s
12:	learn: 0.2504768	total: 5.92s	remaining: 1m 25s
13:	learn: 0.2451690	total: 6.33s	remaining: 1m 24s
14:	learn: 0.2407061	total: 6.78s	remaining: 1m 23s
15:	learn: 0.2370670	total: 7.21s	remaining: 1m 22s
16:	learn: 0.2337753	total: 7.65s	remaining: 1m 22s
17:	learn: 0.2301996	total: 8.07s	remaining: 1m 21s
18:	learn: 0.2277738	total: 8.51s	remaining: 1m 21s
19:	learn: 0.2256222	tot

0:	learn: 0.5917728	total: 532ms	remaining: 1m 45s
1:	learn: 0.5143287	total: 906ms	remaining: 1m 29s
2:	learn: 0.4550757	total: 1.29s	remaining: 1m 24s
3:	learn: 0.4070732	total: 1.65s	remaining: 1m 20s
4:	learn: 0.3698134	total: 2.17s	remaining: 1m 24s
5:	learn: 0.3416777	total: 2.59s	remaining: 1m 23s
6:	learn: 0.3188141	total: 3.06s	remaining: 1m 24s
7:	learn: 0.3010763	total: 3.5s	remaining: 1m 23s
8:	learn: 0.2867814	total: 3.89s	remaining: 1m 22s
9:	learn: 0.2752029	total: 4.31s	remaining: 1m 21s
10:	learn: 0.2663764	total: 4.75s	remaining: 1m 21s
11:	learn: 0.2588749	total: 5.2s	remaining: 1m 21s
12:	learn: 0.2527008	total: 5.64s	remaining: 1m 21s
13:	learn: 0.2472238	total: 6.07s	remaining: 1m 20s
14:	learn: 0.2418211	total: 6.58s	remaining: 1m 21s
15:	learn: 0.2382449	total: 7.02s	remaining: 1m 20s
16:	learn: 0.2349116	total: 7.46s	remaining: 1m 20s
17:	learn: 0.2319445	total: 7.88s	remaining: 1m 19s
18:	learn: 0.2296488	total: 8.32s	remaining: 1m 19s
19:	learn: 0.2266696	tot

16:	learn: 0.2232807	total: 37.6s	remaining: 6m 45s
17:	learn: 0.2204074	total: 39.6s	remaining: 6m 40s
18:	learn: 0.2178236	total: 41.6s	remaining: 6m 36s
19:	learn: 0.2154877	total: 43.3s	remaining: 6m 29s
20:	learn: 0.2135501	total: 44.9s	remaining: 6m 23s
21:	learn: 0.2111830	total: 46.7s	remaining: 6m 18s
22:	learn: 0.2095027	total: 48.5s	remaining: 6m 12s
23:	learn: 0.2073312	total: 50.6s	remaining: 6m 11s
24:	learn: 0.2058270	total: 52.6s	remaining: 6m 8s
25:	learn: 0.2045258	total: 54.6s	remaining: 6m 5s
26:	learn: 0.2033301	total: 56.7s	remaining: 6m 3s
27:	learn: 0.2019817	total: 59.6s	remaining: 6m 5s
28:	learn: 0.2007843	total: 1m 1s	remaining: 6m 3s
29:	learn: 0.1997820	total: 1m 4s	remaining: 6m 4s
30:	learn: 0.1983082	total: 1m 6s	remaining: 6m 5s
31:	learn: 0.1973554	total: 1m 9s	remaining: 6m 6s
32:	learn: 0.1964036	total: 1m 12s	remaining: 6m 5s
33:	learn: 0.1955769	total: 1m 14s	remaining: 6m 6s
34:	learn: 0.1946914	total: 1m 17s	remaining: 6m 4s
35:	learn: 0.1931959

0:	learn: 0.5882231	total: 1.83s	remaining: 6m 4s
1:	learn: 0.5088497	total: 4.37s	remaining: 7m 12s
2:	learn: 0.4458113	total: 6.74s	remaining: 7m 22s
3:	learn: 0.3956836	total: 9.09s	remaining: 7m 25s
4:	learn: 0.3587926	total: 11.5s	remaining: 7m 27s
5:	learn: 0.3285603	total: 13.9s	remaining: 7m 31s
6:	learn: 0.3041528	total: 16.4s	remaining: 7m 30s
7:	learn: 0.2862150	total: 18.8s	remaining: 7m 30s
8:	learn: 0.2715365	total: 21.1s	remaining: 7m 28s
9:	learn: 0.2578032	total: 23.5s	remaining: 7m 27s
10:	learn: 0.2482758	total: 26s	remaining: 7m 27s
11:	learn: 0.2406134	total: 28.5s	remaining: 7m 27s
12:	learn: 0.2344206	total: 31s	remaining: 7m 26s
13:	learn: 0.2288956	total: 33.4s	remaining: 7m 24s
14:	learn: 0.2243157	total: 35.8s	remaining: 7m 21s
15:	learn: 0.2197235	total: 38.3s	remaining: 7m 20s
16:	learn: 0.2165920	total: 40.7s	remaining: 7m 18s
17:	learn: 0.2136641	total: 43.1s	remaining: 7m 15s
18:	learn: 0.2109380	total: 45.4s	remaining: 7m 12s
19:	learn: 0.2084228	total:

103:	learn: 0.1515722	total: 4m 8s	remaining: 3m 49s
104:	learn: 0.1513188	total: 4m 11s	remaining: 3m 47s
105:	learn: 0.1510503	total: 4m 13s	remaining: 3m 44s
106:	learn: 0.1506238	total: 4m 15s	remaining: 3m 42s
107:	learn: 0.1502528	total: 4m 18s	remaining: 3m 39s
108:	learn: 0.1499513	total: 4m 20s	remaining: 3m 37s
109:	learn: 0.1497104	total: 4m 22s	remaining: 3m 35s
110:	learn: 0.1493965	total: 4m 25s	remaining: 3m 32s
111:	learn: 0.1491369	total: 4m 27s	remaining: 3m 30s
112:	learn: 0.1488885	total: 4m 29s	remaining: 3m 27s
113:	learn: 0.1486012	total: 4m 32s	remaining: 3m 25s
114:	learn: 0.1482827	total: 4m 34s	remaining: 3m 22s
115:	learn: 0.1480225	total: 4m 37s	remaining: 3m 20s
116:	learn: 0.1477377	total: 4m 39s	remaining: 3m 18s
117:	learn: 0.1474594	total: 4m 41s	remaining: 3m 15s
118:	learn: 0.1471696	total: 4m 44s	remaining: 3m 13s
119:	learn: 0.1469312	total: 4m 46s	remaining: 3m 10s
120:	learn: 0.1467285	total: 4m 48s	remaining: 3m 8s
0:	learn: 0.5929375	total: 698

121:	learn: 0.1464901	total: 4m 50s	remaining: 3m 6s
122:	learn: 0.1462322	total: 4m 53s	remaining: 3m 3s
123:	learn: 0.1459857	total: 4m 55s	remaining: 3m 1s
124:	learn: 0.1457734	total: 4m 57s	remaining: 2m 58s
125:	learn: 0.1455801	total: 5m	remaining: 2m 56s
126:	learn: 0.1453518	total: 5m 2s	remaining: 2m 53s
127:	learn: 0.1450261	total: 5m 4s	remaining: 2m 51s
128:	learn: 0.1448172	total: 5m 6s	remaining: 2m 48s
129:	learn: 0.1445513	total: 5m 9s	remaining: 2m 46s
130:	learn: 0.1443230	total: 5m 11s	remaining: 2m 44s
131:	learn: 0.1441280	total: 5m 13s	remaining: 2m 41s
132:	learn: 0.1438790	total: 5m 15s	remaining: 2m 39s
133:	learn: 0.1437115	total: 5m 18s	remaining: 2m 36s
134:	learn: 0.1434774	total: 5m 20s	remaining: 2m 34s
135:	learn: 0.1431903	total: 5m 22s	remaining: 2m 31s
136:	learn: 0.1430276	total: 5m 25s	remaining: 2m 29s
137:	learn: 0.1428295	total: 5m 27s	remaining: 2m 27s
138:	learn: 0.1425239	total: 5m 29s	remaining: 2m 24s
139:	learn: 0.1423110	total: 5m 31s	rem

(<catboost.core.CatBoostClassifier at 0x16bec4400>, 0.7219725835179578)

In [33]:
result = pd.DataFrame(scores_data)
result.columns = ['Model', 'F1', 'Fit_time', 'Score_time', 'Data', 'Param']
result.set_index('F1', inplace = True)
display(result.sort_values('F1', ascending = False))

,Model,Fit_time,Score_time,Data,Param
F1,,,,,
0.7733,LGBMClassifier(),16.037,0.645,text_wnl_postag,"{'class_weight': {0: 1, 1: 8.841344371679229},..."
0.7719,LGBMClassifier(),16.002,0.540,text_wnl,"{'class_weight': {0: 1, 1: 8.841344371679229},..."
0.7688,LogisticRegression(),4.644,0.004,text_wnl,"{'C': 5, 'class_weight': {0: 1, 1: 8.841344371..."
0.7651,LogisticRegression(),1.206,0.004,text_wnl_postag,"{'C': 5, 'class_weight': {0: 1, 1: 8.841344371..."
0.7612,SGDClassifier(),1.193,0.003,text_wnl,"{'class_weight': {0: 1, 1: 8.841344371679229},..."
0.7589,SGDClassifier(),1.083,0.004,text_wnl_postag,"{'class_weight': {0: 1, 1: 8.841344371679229},..."
0.7220,<catboost.core.CatBoostClassifier object at 0x...,2618.292,0.344,text_wnl_postag,"{'depth': 9, 'iterations': 200, 'learning_rate..."
0.7190,<catboost.core.CatBoostClassifier object at 0x...,2893.829,0.460,text_wnl,"{'depth': 9, 'iterations': 200, 'learning_rate..."
0.6500,DecisionTreeClassifier(),23.717,0.016,text_wnl_postag,"{'class_weight': {0: 1, 1: 8.841344371679229},..."


**Вывод:**

В этом разделе выполнено:

Обучены пять моделей:
- LogisticRegression
- DecisionTreeClassifier
- LGBMClassifier
- SGDClassifier
- CatBoostClassifier

без учёта части речи и с учётом части речи (`text_wnl` или `text_wnl_postag`).

Модель LGBMClassifier показала лучший результат по метрике f1 = 0.7733 на данных,
лемматизированных с учетом части речи и с набором параметров:

- 'class_weight': {0: 1, 1: 8.841344371679229}
- 'learning_rate': 0.3
- 'max_depth': 11
- 'n_estimators': 350, 'objective': 'binary'}

Модели LogisticRegression (f1 = 0.7688) и SGDClassifier (f1 = 0.7612) показали хорошие результаты по метрике f1  на данных, лемматизированных без учета части речи, при этом время обучения и предсказания значительно меньше. Если выбирать по совокупности факторов (f1, fi_time, score_time) можно рассмотретть эти модели.

<a id=5></a>
[Содержание](#0)
## 5. Тестирование модели

Проверим качество LGBMClassifier на тестовой выборке.

In [35]:
model = LGBMClassifier(n_estimators = 350, class_weight = {0 : 1, 1 : class_ratio},
                     boosting_type = 'gbdt', learning_rate = 0.3, max_depth = 11, objective = 'binary')
model.fit(tf_idf_tag_train, y_train)
predicted = model.predict(tf_idf_tag_test)
f1 = f1_score(y_test, predicted)
print('Значение f1 на тестовых данных:', f1)

Значение f1 на тестовых данных: 0.7770569153641993


**Вывод:**

В результате выявлено следующее:

- Наилучшая модель **LGBMClassifier** (на лемматизированных данных с учетом части речи и с параметрами {'class_weight': {0: 1, 1: 8.841344371679229}, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 350, 'objective': 'binary'} на тестовой выборке имеет метрику **f1 = 0.777**.
- Значение метрики **f1 на тестовой выборке не меньше 0.75**, что соответствует условиям задачи.

<a id=6></a>
[Содержание](#0)
## 6. Общий вывод

Проведено исследование с целью обучить модель классифицировать комментарии на позитивные и негативные.

Входные данные - набор данных с разметкой о токсичности правок.

Наилучшая модель LGBMClassifier (на лемматизированных данных с учетом части речи) на тестовой выборке имеет метрику f1 = 0.777.

Значение метрики f1 на тестовой выборке не меньше 0.75, что соответствует условиям задачи.

Магазину можно рекомендовать использовать полученную модель **LGBMClassifier** в качестве инструмента, который будет определять токсичные комментарии и отправлять их на модерацию.